In [2]:
# モデルハイパーパラメータ： 非表示レイヤーの数と幅などのモデルの選択に影響
# アルゴリズムハイパーパラメータ: 確率勾配降下法（SGD)の学習速度やk最近傍(kNN)分類機の最近傍の数など、学習アルゴリズムの速度と質に影響
import tensorflow as tf
from tensorflow import keras

import IPython

In [15]:
!pip install -U keras-tuner
import kerastuner as kt

Requirement already up-to-date: keras-tuner in /Users/mugaikotsu/miniforge3/envs/m1_mac_tensorflow/lib/python3.8/site-packages (1.1.2)


In [16]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [17]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [20]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [22]:
# チューナーをインスタンス化してハイパーチューニングを実行
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [23]:
# トレーニングステップごとにトレーニング出力をクリアにするコールバックを定義
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [24]:
# ハイパーパラメータ検索を実行
tuner.search(img_train, label_train, epochs=10, validation_data=(img_test, label_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 27s]
val_accuracy: 0.8507000207901001

Best val_accuracy So Far: 0.8834999799728394
Total elapsed time: 00h 04m 27s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 128 and the optimal learning rate for the optimizer
is 0.001.



In [25]:
# 検索から得た最適なハイパーパラメータでモデルを保存する
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 2s 944us/step - loss: 0.4972 - accuracy: 0.8260 - val_loss: 0.4230 - val_accuracy: 0.8487
Epoch 2/10
1875/1875 [==============================] - 2s 915us/step - loss: 0.3732 - accuracy: 0.8658 - val_loss: 0.3787 - val_accuracy: 0.8647
Epoch 3/10
1875/1875 [==============================] - 2s 899us/step - loss: 0.3377 - accuracy: 0.8785 - val_loss: 0.3581 - val_accuracy: 0.8754
Epoch 4/10
1875/1875 [==============================] - 2s 837us/step - loss: 0.3114 - accuracy: 0.8857 - val_loss: 0.3562 - val_accuracy: 0.8743
Epoch 5/10
1875/1875 [==============================] - 2s 860us/step - loss: 0.2926 - accuracy: 0.8915 - val_loss: 0.3517 - val_accuracy: 0.8774
Epoch 6/10
1875/1875 [==============================] - 2s 892us/step - loss: 0.2779 - accuracy: 0.8974 - val_loss: 0.3435 - val_accuracy: 0.8787
Epoch 7/10
1875/1875 [==============================] - 2s 807us/step - loss: 0.2642 - accuracy: 0.9015 - val_loss: 0.3367 -